In [1]:
import warnings
warnings.filterwarnings("ignore")

from carla.data.catalog import OnlineCatalog
from carla import MLModelCatalog
from carla.recourse_methods import Clue, Wachter
from carla.models.negative_instances import predict_negative_instances
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from time import perf_counter

num = 10
data_name = "compas"

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [ ]:
def train_new_model(dataset):
    model = MLModelCatalog(dataset, "ann", backend="pytorch")
    model.train(
        learning_rate = 0.001,
        epochs = 10,
        max_depth = 50,
        n_estimators = 50,
        batch_size = 20,
        force_train = True
    )
    return model

In [ ]:
def update_dataset(dataset, factuals, counterfactuals):
    fac_ind = []
#     for index, row in factuals.iterrows():
#         fac_ind.append(index)
#     for index, row in counterfactuals.iterrows():
#         dataset.loc[index] = counterfactuals.loc[index]
        
    for ((i_f, r_f), (i_c, r_c)) in zip(factuals.iterrows(), counterfactuals.iterrows()):
        if len(counterfactuals.loc[i_c].dropna()) > 0:
            dataset.loc[i_f] = counterfactuals.loc[i_c]

In [ ]:
def train_recourse_method(method):
    rm = None
    if method == "clue":
        hyperparams = {
                "data_name": data_name,
                "train_vae": True,
                "width": 10,
                "depth": 3,
                "latent_dim": 12,
                "batch_size": 64,
                "epochs": 1,
                "lr": 0.001,
                "early_stop": 20,
            }

        # load a recourse model and pass black box model
        rm = Clue(dataset, model, hyperparams)
        
    else:
        hyperparams = {
                "loss_type": "BCE"
            }

        # load a recourse model and pass black box model
        rm = Wachter(model, hyperparams)
        
        
    return rm

In [ ]:
def predict(model, data):
    pred = model.predict(data._df)
    return np.where(pred > 0.5, 1, 0)

In [ ]:
def print_f1_score(model, data):
    score = f1_score(np.array(data._df[data.target]), predict(model, data))
    print(f"F1 score: {score}")

In [ ]:
def print_accuracy(model, data):
    score = accuracy_score(np.array(data._df[data.target]), predict(model, data))
    print(f"Accuracy score: {score}")

In [ ]:
def print_scores(model_pre, model_post, data):
    print("Before recourse:")
    print_f1_score(model_pre, data)
    print_accuracy(model_pre, data)
    print("\nAfter recourse:")
    print_f1_score(model_post, data)
    print_accuracy(model_post, data)

In [2]:
# load the dataset
dataset = OnlineCatalog(data_name)

# train a model on the dataset
model = train_new_model(dataset)

# generate counterfactual samples
factuals = predict_negative_instances(model, dataset._df).sample(num)
print("Number of factuals", len(factuals))

pre = model.predict(factuals)

NameError: name 'train_new_model' is not defined

In [ ]:
rm = train_recourse_method("wachter")

In [ ]:
counterfactuals = rm.get_counterfactuals(factuals)
print("Number of counterfactuals:", len(counterfactuals.dropna()))

In [ ]:
# load a new dataset
d_c = OnlineCatalog(data_name)

# replace factuals with counterfactuals
update_dataset(d_c._df, factuals, counterfactuals)

# train the new model
model2 = train_new_model(d_c)

In [ ]:
print_scores(model, model2, dataset)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(factuals['age'], factuals['length_of_stay'], c=factuals['score'], marker='o')

plt.scatter(counterfactuals['age'], counterfactuals['length_of_stay'], c=counterfactuals['score'], marker='s')
plt.show()

In [ ]:
factuals.index

In [ ]:
counterfactuals.index

In [3]:
df = dataset.df

In [6]:
df[df.columns.values[0]].max()

1.0